In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("moltean/fruits")

print("Path to dataset files:", path)

100%|██████████| 964M/964M [21:59<00:00, 766kB/s]   

Extracting files...


Path to dataset files: C:\Users\BineshMahato\.cache\kagglehub\datasets\moltean\fruits\versions\11


In [23]:
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the paths to your train and validation directories
train_dir = r'C:\Users\BineshMahato\.cache\kagglehub\datasets\moltean\fruits\versions\11\fruits-360_dataset_100x100\fruits-360\Test'
classes = os.listdir(train_dir)
print(f"Total subdirectories (classes): {len(classes)}")
print("Classes found:", classes)
val_dir = r'C:\Users\BineshMahato\.cache\kagglehub\datasets\moltean\fruits\versions\11\fruits-360_dataset_100x100\fruits-360\Test'
# Image preprocessing for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=40,  # Randomly rotate images
    width_shift_range=0.2,  # Randomly shift horizontally
    height_shift_range=0.2,  # Randomly shift vertically
    shear_range=0.2,  # Shear angle
    zoom_range=0.2,  # Random zoom
    horizontal_flip=True,  # Random horizontal flip
    fill_mode='nearest'  # Fill pixels after transformations
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# Create training and validation generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Resize images to 150x150
    batch_size=32,
    class_mode='categorical'  # Multi-class classification
)
print(f"Number of classes: {len(train_generator.class_indices)}")


validation_generator = validation_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)


Total subdirectories (classes): 141
Classes found: ['Apple 6', 'Apple Braeburn 1', 'Apple Crimson Snow 1', 'Apple Golden 1', 'Apple Golden 2', 'Apple Golden 3', 'Apple Granny Smith 1', 'Apple hit 1', 'Apple Pink Lady 1', 'Apple Red 1', 'Apple Red 2', 'Apple Red 3', 'Apple Red Delicious 1', 'Apple Red Yellow 1', 'Apple Red Yellow 2', 'Apricot 1', 'Avocado 1', 'Avocado ripe 1', 'Banana 1', 'Banana Lady Finger 1', 'Banana Red 1', 'Beetroot 1', 'Blueberry 1', 'Cabbage white 1', 'Cactus fruit 1', 'Cantaloupe 1', 'Cantaloupe 2', 'Carambula 1', 'Carrot 1', 'Cauliflower 1', 'Cherry 1', 'Cherry 2', 'Cherry Rainier 1', 'Cherry Wax Black 1', 'Cherry Wax Red 1', 'Cherry Wax Yellow 1', 'Chestnut 1', 'Clementine 1', 'Cocos 1', 'Corn 1', 'Corn Husk 1', 'Cucumber 1', 'Cucumber 3', 'Cucumber Ripe 1', 'Cucumber Ripe 2', 'Dates 1', 'Eggplant 1', 'Eggplant long 1', 'Fig 1', 'Ginger Root 1', 'Granadilla 1', 'Grape Blue 1', 'Grape Pink 1', 'Grape White 1', 'Grape White 2', 'Grape White 3', 'Grape White 4', 

In [24]:
from tensorflow.keras import layers, models

# Initialize the model
vegetable_model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(len(train_generator.class_indices), activation='softmax')  # Number of classes
])

# Compile the model
vegetable_model.compile(optimizer='adam',
                        loss='categorical_crossentropy',
                        metrics=['accuracy'])


In [25]:
history = vegetable_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)


Epoch 1/10
738/738 [==============================] - 372s 504ms/step - loss: 1.7283 - accuracy: 0.5287 - val_loss: 0.5077 - val_accuracy: 0.8462
Epoch 2/10
738/738 [==============================] - 371s 502ms/step - loss: 0.4214 - accuracy: 0.8574 - val_loss: 0.2570 - val_accuracy: 0.9220
Epoch 3/10
738/738 [==============================] - 382s 517ms/step - loss: 0.2540 - accuracy: 0.9150 - val_loss: 0.0957 - val_accuracy: 0.9672
Epoch 4/10
738/738 [==============================] - 373s 505ms/step - loss: 0.2221 - accuracy: 0.9276 - val_loss: 0.0925 - val_accuracy: 0.9638
Epoch 5/10
738/738 [==============================] - 383s 518ms/step - loss: 0.1609 - accuracy: 0.9479 - val_loss: 0.1428 - val_accuracy: 0.9584
Epoch 6/10
738/738 [==============================] - 413s 560ms/step - loss: 0.1565 - accuracy: 0.9491 - val_loss: 0.1270 - val_accuracy: 0.9645
Epoch 7/10
738/738 [==============================] - 390s 529ms/step - loss: 0.1558 - accuracy: 0.9519 - val_loss: 0.1616 -

In [30]:
# Evaluate on validation data
val_loss, val_acc = vegetable_model.evaluate(validation_generator)
print(f'Validation accuracy: {val_acc * 100:.2f}%')


146/739 [====>.........................] - ETA: 43s - loss: 0.1147 - accuracy: 0.9653

KeyboardInterrupt: 

In [32]:
import numpy as np
from tensorflow.keras.preprocessing import image

def prepare_image(img_path):
    img = image.load_img(img_path, target_size=(150, 150))  # Resize to match model input
    img_array = image.img_to_array(img)  # Convert image to numpy array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize pixel values to [0, 1]
    return img_array

# Example prediction
img_path = 'images4.jpg'  # Replace with path to your test image
img_array = prepare_image(img_path)

# Predict the class
predictions = vegetable_model.predict(img_array)
predicted_class = np.argmax(predictions, axis=1)
print(f'Predicted class: {list(train_generator.class_indices.keys())[predicted_class[0]]}')


1/1 [==============================] - 0s 20ms/step
Predicted class: Banana Lady Finger 1


In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()
